# Explaining the Code


1. Optimization Phase: For a given $\tilde \gamma$ we solve the problem, using some sort of optimization algorithm (IPOPT in our case),  

\begin{equation}
      \left\{ \int_0^\infty \exp(-\delta t) \left[-P^e  \left (\kappa\sum_{i=1}^I Z^i_t- \sum_{i=1}^I \dot X^i_t \right)+  P^a_t  \sum_i \theta^i Z^i_t-\frac \zeta 2 \left (\sum_i U_t^i + V_t^i \right)^2 \right ] dt\right\} 
\end{equation}

\begin{equation} \label{eq:z}
\dot Z_t^i = U_t^i - V_t^i . 
\end{equation}

\begin{equation} \label{eq:x}
{\dot X}_t^i  = - \tilde \gamma^i U^i_t - \alpha \left[ X_t^i - \tilde \gamma^i  \left( {{\bar z}^i - Z_t^i }  \right) \right] 
\end{equation}

$$
X_0^i = \tilde \gamma^i * C
$$

    where $C$ is some constant. 

2. MC phase: The algorithm for this part is the standard Metropolis-Hastings Algorirthm. The only thing that is a bit troublesome to deal with is the formulation of our likelihood which is $g$. 

    1. We are given $\gamma^*$ via the MHMC algorithm. 
    2. Use $U_t^i$ and $V_t^i$ to evaluate the objective function under $\gamma^*$. 
    3. Form the Likelihood $g$

\begin{equation}\label{min_solution}
g^* = \exp\left[ - {\frac 1 \xi } \left\{ \int_0^\infty \exp(-\delta t) \left[-P^e  \left (\kappa\sum_{i=1}^I Z^i_t- \sum_{i=1}^I \dot X^i_t \right)+  P^a_t  \sum_i \theta^i Z^i_t-\frac \zeta 2 \left (\sum_i U_t^i + V_t^i \right)^2 \right ] dt\right\} \right]
\end{equation} 
    

In [ ]:
#!/usr/bin/env python

# Import Required Packages
# ========================
import os, sys
import pickle
import time

import casadi
import numpy as np
import matplotlib.pyplot as plt

# MCMC (HMC) sampling routines
sys.path.append(os.path.abspath("mcmc"))
from mcmc_sampling import create_hmc_sampler

import mcmc_with_casadi
# import mcmc_with_gams

# Local Debugging flag; remove when all tested
_DEBUG = False 

## Start Running the test

In [ ]:
# Call the main interface `main` with default settings
casadi_results = mcmc_with_casadi.main()
# gams_results = mcmc_with_gams.main()

### Plot Results

In [ ]:
# Plot Error Results
plt.plot(results['error_tracker'])
plt.xlabel("Iteration")
plt.ylabel("Error")
plt.show()

In [ ]:
# Plot Gamma Estimate Update
for j in range(results['gamma_size']):
    plt.plot(results['gamma_vals_tracker'][:, j], label=r"$\gamma_{%d}$"%(j+1))
plt.legend(bbox_to_anchor=(1.04, 0.5), ncol=2, loc="center left", borderaxespad=0)
plt.show()

In [ ]:
# Plot Histograms
for itr in results['collected_ensembles'].keys():
    for j in range(results['gamma_size']):
        plt.hist(results['collected_ensembles'][itr][:, j], bins=100)
        plt.title(f"Iteration {itr+1}; Site {j+1}")
        plt.show()

In [ ]:
# Plot Histogram of the final sample
for j in range(results['gamma_size']):
    plt.hist(results['final_sample'][:, j], bins=100)
    plt.title(f"Final Sample; Site {j+1}")
    plt.show()